## section 1: derivatives

In [17]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
    return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [18]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29


def gradf(a, b, c):
  derivative_a = -3*a**2 - 0.5*a**-0.5
  derivative_b = 3*cos(3*b) + 2.5*b**1.5
  derivative_c = 1.0 / c**2
  return [derivative_a, derivative_b, derivative_c]

# Test the gradient function
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")



OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [27]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

def numerical_gradient(f, a, b, c, h=1e-5):

  derivative_a = (f(a + h, b, c) - f(a , b, c)) / h
  derivative_b = (f(a, b + h, c) - f(a, b , c)) / h
  derivative_c = (f(a, b, c + h) - f(a, b, c )) / h
  return [derivative_a,derivative_b,derivative_c]

# Test the numerical gradient
numerical_grad = numerical_gradient(f, 2, 3, 4)

ans = [-12.353553390593273, 10.25699027111255, 0.0625]
for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")



WRONG! for dim 0: expected -12.353553390593273, yours returns -12.353612948956536
WRONG! for dim 1: expected 10.25699027111255, yours returns 10.257004202163245
OK for dim 2: expected 0.0625, yours returns 0.06249984378925432


In [28]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.



def numerical_gradient2(f, a, b, c, h=1e-5):

   derivative_a= (f(a + h, b, c) - f(a - h, b, c)) / (2 * h)
   derivative_b= (f(a, b + h, c) - f(a, b - h, c)) / (2 * h)
   derivative_c= (f(a, b, c + h) - f(a, b, c - h)) / (2 * h)
   return [derivative_a,derivative_b,derivative_c]

numerical_grad2 = numerical_gradient2(f, 2, 3, 4)

ans = [-12.353553390593273, 10.25699027111255, 0.0625]
for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")



OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


## section 2: support for softmax

In [29]:
from math import exp, log

class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __radd__(self, other):
    return self + other

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other):
    return self * other

  def __sub__(self, other):
    return self + (-other)

  def __neg__(self):
    return self * -1

  def __truediv__(self, other):
    return self * other**-1

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data ** other, (self,), f'**{other}')

    def _backward():
      self.grad += (other * self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    out = Value(exp(self.data), (self,), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def log(self):
    out = Value(log(self.data), (self,), 'log')

    def _backward():
      self.grad += (1 / self.data) * out.grad
    out._backward = _backward

    return out

  def backward(self):
    topo = []
    visited = set()

    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)

    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

  def zero_grad(self):
    self.grad = 0.0
    for child in self._prev:
      child.zero_grad()

def softmax(logits):
  exp_values = [logit.exp() for logit in logits]
  sum_exp_values = sum(exp_values)
  out = [exp_val / sum_exp_values for exp_val in exp_values]

  def _backward():
    for i in range(len(logits)):
      logits[i].grad += out[i].data * (1 - out[i].data) * out[i].grad
      for j in range(len(logits)):
        if i != j:
          logits[i].grad -= out[i].data * out[j].data * out[j].grad

  for val in out:
    val._backward = _backward

  return out

# Test the corrected implementation
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]

probs = softmax(logits)

loss = -probs[3].log()

loss.backward()



ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633


In [22]:
import torch

logits_torch = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)
softmax_torch = torch.nn.functional.softmax(logits_torch, dim=0)
loss_torch = -torch.log(softmax_torch[3])  # dim 3 acts as the label for this input example
loss_torch.backward()

print("Loss:", loss_torch.item())  # Output the loss value

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits_torch.grad[dim].item() - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, torch returns {logits_torch.grad[dim].item()}")

print("\nGradient comparison:")
for i in range(4):
  print(f"Dimension {i}:")
  print(f"Custom grad: {logits[i].grad}")
  print(f"Torch grad: {logits_torch.grad[i].item()}")


Loss: 2.1755154132843018
OK for dim 0: expected 0.041772570515350445, torch returns 0.041772566735744476
OK for dim 1: expected 0.8390245074625319, torch returns 0.8390244841575623
OK for dim 2: expected 0.005653302662216329, torch returns 0.005653302650898695
OK for dim 3: expected -0.8864503806400986, torch returns -0.8864504098892212

Gradient comparison:
Dimension 0:
Custom grad: 0.041772570515350445
Torch grad: 0.041772566735744476
Dimension 1:
Custom grad: 0.8390245074625319
Torch grad: 0.8390244841575623
Dimension 2:
Custom grad: 0.00565330266221633
Torch grad: 0.005653302650898695
Dimension 3:
Custom grad: -0.8864503806400986
Torch grad: -0.8864504098892212


In [99]:
import os
os.environ["PATH"] += os.pathsep + 'C:\Program Files\Graphviz\bin'